In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np

# Loading the dataset from the URL
url = 'https://data.cityofchicago.org/resource/dqcy-ctma.csv?$limit=5000000'
df = pd.read_csv(url)

# Displaying basic information about the dataset
print("Dataset Information:")
df.info()

# Displaying the first 5 rows of the dataset
print("\nFirst 5 rows of the dataset:")
print(df.head())

# Handling missing values before processing
print("\nMissing values before cleaning:")
print(df.isnull().sum())

# Dropping duplicates
df.drop_duplicates(inplace=True)

# Converting 'date' column to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Keeping only relevant columns: longitude, latitude, and primary type of crime
df = df[['longitude', 'latitude', 'primary_type']]

# Dropping rows with missing longitude or latitude
df = df.dropna(subset=['longitude', 'latitude'])

# Identifying the top 10 most common crime types
top_10_crimes = df['primary_type'].value_counts().nlargest(10).index

# Filtering the dataset to include only rows with the top 10 crime types
df = df[df['primary_type'].isin(top_10_crimes)]

feature_type_to_id = {ftype: idx for idx, ftype in enumerate(df['primary_type'].unique(), start=0)}
df['feature_id'] = df['primary_type'].map(feature_type_to_id)

# Resetting the index after filtering
df.reset_index(drop=True, inplace=True)

# Final check for missing values
print("\nMissing values after cleaning:")
print(df.isnull().sum())

# Displaying summary statistics after cleaning
print("\nSummary statistics:")
print(df.describe())

# Displaying the top 5 rows after cleaning
print("\nTop 5 rows after cleaning:")
print(df.head())

print("\n Top 10 crime types:")
print(top_10_crimes)

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193074 entries, 0 to 193073
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    193074 non-null  int64  
 1   case_number           193074 non-null  object 
 2   date                  193074 non-null  object 
 3   block                 193074 non-null  object 
 4   iucr                  193074 non-null  object 
 5   primary_type          193074 non-null  object 
 6   description           193074 non-null  object 
 7   location_description  192313 non-null  object 
 8   arrest                193074 non-null  bool   
 9   domestic              193074 non-null  bool   
 10  beat                  193074 non-null  int64  
 11  district              193074 non-null  int64  
 12  ward                  193074 non-null  int64  
 13  community_area        193074 non-null  int64  
 14  fbi_code              193074 no

In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

shapefile = '/home/amk7r/colocation_mining/parallel_project/data/chicago/shapefiles'
gdf_polygon = gpd.read_file(shapefile)

# Step 4: Ensure the coordinate reference systems match (if not, reproject)
gdf_points = gdf_points.set_crs(epsg=4326)  # Assuming points are in WGS84 (lat/long)
gdf_polygon = gdf_polygon.to_crs(gdf_points.crs)

# Step 5: Perform spatial join or intersection to filter points within the polygon
points_in_polygon = gdf_points[gdf_points.geometry.within(gdf_polygon.unary_union)]

# Step 6: Resulting points
print(points_in_polygon)

/tmp/ipykernel_8076/1681794452.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  points_in_polygon = gdf_points[gdf_points.geometry.within(gdf_polygon.unary_union)]


        longitude   latitude        primary_type  feature_id  \
0      -87.658045  41.884110     CRIMINAL DAMAGE           0   
1      -87.766025  41.924902             BATTERY           1   
2      -87.627513  41.899151               THEFT           2   
3      -87.752759  41.801166  DECEPTIVE PRACTICE           3   
4      -87.702220  41.743549  DECEPTIVE PRACTICE           3   
...           ...        ...                 ...         ...   
178513 -87.677273  41.998916               THEFT           2   
178514 -87.743655  41.884474       OTHER OFFENSE           6   
178515 -87.773525  41.887980     CRIMINAL DAMAGE           0   
178516 -87.640890  41.907045  DECEPTIVE PRACTICE           3   
178517 -87.657582  41.804993               THEFT           2   

                          geometry  
0       POINT (-87.65805 41.88411)  
1        POINT (-87.76603 41.9249)  
2       POINT (-87.62751 41.89915)  
3       POINT (-87.75276 41.80117)  
4       POINT (-87.70222 41.74355)  
...      

In [3]:
selected_columns = points_in_polygon[['feature_id', 'latitude', 'longitude']]
selected_columns.to_csv('/home/amk7r/colocation_mining/parallel_project/data/chicago/chicago.csv', index=False)

In [4]:
print('Size of Chicago Data:', len(selected_columns))

Size of Chicago Data: 178480


In [6]:
# Importing necessary libraries
import pandas as pd
import numpy as np

# Loading the dataset from the URL
url = 'https://data.seattle.gov/resource/tazs-3rd5.csv?$limit=5000000'
df = pd.read_csv(url)

# Displaying basic information about the dataset
print("Dataset Information:")
df.info()

# Displaying the first 5 rows of the dataset
print("\nFirst 5 rows of the dataset:")
print(df.head())

# Handling missing values before processing
print("\nMissing values before cleaning:")
print(df.isnull().sum())

# Dropping duplicates
df.drop_duplicates(inplace=True)

# Converting datetime columns to datetime objects
df['report_datetime'] = pd.to_datetime(df['report_datetime'], errors='coerce')
df['offense_start_datetime'] = pd.to_datetime(df['offense_start_datetime'], errors='coerce')

# Handling missing values in datetime columns
df = df[df['report_datetime'].notnull()]
df = df[df['offense_start_datetime'].notnull()]

# Keeping only relevant columns: longitude, latitude, and offense type
df = df[['longitude', 'latitude', 'offense']]

# Dropping rows with missing longitude or latitude
df = df.dropna(subset=['longitude', 'latitude'])

# Identifying the top 10 most common crime types (offense)
top_10_crimes = df['offense'].value_counts().nlargest(10).index

# Filtering the dataset to include only rows with the top 10 crime types
df = df[df['offense'].isin(top_10_crimes)]

feature_type_to_id = {ftype: idx for idx, ftype in enumerate(df['offense'].unique(), start=0)}
df['feature_id'] = df['offense'].map(feature_type_to_id)

# Resetting the index after filtering
df.reset_index(drop=True, inplace=True)

# Final check for missing values
print("\nMissing values after cleaning:")
print(df.isnull().sum())

# Displaying summary statistics after cleaning
print("\nSummary statistics:")
print(df.describe())

# Displaying the top 5 rows after cleaning
print("\nTop 5 rows after cleaning:")
print(df.head())

print("\n Top 10 crime types:")
print(top_10_crimes)

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149893 entries, 0 to 1149892
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   report_number           1149893 non-null  object 
 1   offense_id              1149893 non-null  int64  
 2   offense_start_datetime  1148010 non-null  object 
 3   offense_end_datetime    662969 non-null   object 
 4   report_datetime         1149893 non-null  object 
 5   group_a_b               1149893 non-null  object 
 6   crime_against_category  1149893 non-null  object 
 7   offense_parent_group    1149893 non-null  object 
 8   offense                 1149893 non-null  object 
 9   offense_code            1149893 non-null  object 
 10  precinct                1149883 non-null  object 
 11  sector                  1149883 non-null  object 
 12  beat                    1149883 non-null  object 
 13  mcpp                    1149862 non-

In [7]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

shapefile = '/home/amk7r/colocation_mining/parallel_project/data/seattle/shapefiles'
gdf_polygon = gpd.read_file(shapefile)

# Step 4: Ensure the coordinate reference systems match (if not, reproject)
gdf_points = gdf_points.set_crs(epsg=4326)  # Assuming points are in WGS84 (lat/long)
gdf_polygon = gdf_polygon.to_crs(gdf_points.crs)

# Step 5: Perform spatial join or intersection to filter points within the polygon
points_in_polygon = gdf_points[gdf_points.geometry.within(gdf_polygon.unary_union)]

# Step 6: Resulting points
print(points_in_polygon)

/tmp/ipykernel_8076/4176840835.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  points_in_polygon = gdf_points[gdf_points.geometry.within(gdf_polygon.unary_union)]


         longitude   latitude                                      offense  \
0      -122.323399  47.675118  Theft of Motor Vehicle Parts or Accessories   
1      -122.384865  47.642927     Destruction/Damage/Vandalism of Property   
2      -122.338450  47.608789                                  Shoplifting   
3      -122.355309  47.680527     Destruction/Damage/Vandalism of Property   
4      -122.367312  47.521028                                  Shoplifting   
...            ...        ...                                          ...   
871029 -122.350586  47.529242                 Burglary/Breaking & Entering   
871030 -122.240770  47.499634                            All Other Larceny   
871031 -122.351731  47.488775                                 Intimidation   
871034 -122.352349  47.508443                          Motor Vehicle Theft   
871035 -122.351365  47.773575                           Aggravated Assault   

        feature_id                     geometry  
0            

In [8]:
selected_columns = points_in_polygon[['feature_id', 'latitude', 'longitude']]
selected_columns.to_csv('/home/amk7r/colocation_mining/parallel_project/data/seattle/seattle.csv', index=False)

In [9]:
print('Size of Seattle Data:', len(selected_columns))

Size of Seattle Data: 850194
